# Score Scenarios

Run the cells below to score the configured scenarios. Any unscored scenarios will be scored.

To re-score a scenario, open its corresponding file geodatabase and delete its `scores` and `scores_summary` table.

In [64]:
if 'ato_tools' in sys.modules:
    import importlib
    importlib.reload(ato)

In [69]:
import sys
import os
import arcpy

import pandas as pd
from arcgis.features import SpatialDataFrame

src = os.path.join(os.path.abspath("."), 'src')
if src not in sys.path:
    sys.path.append(src)
    
from ato_tools import ato

if 'ato_tools' in sys.modules:
    import importlib
    importlib.reload(ato)

base_path = os.path.abspath(".")

baseline_gdb = "baseline.gdb"

In [70]:
modes = list()
for file in os.listdir(r'scenario'):
    d = os.path.join(base_path, r'scenario', file)
    if os.path.isdir(d) and not d.endswith('.gdb'):
        modes.append(file)

to_score = list()
for mode in ['Cycling', 'Driving', 'Transit']:
    scenario_folder = os.path.join('scenario', mode)
    for file in os.listdir(scenario_folder):
        d = os.path.join(base_path, scenario_folder, file)
        if os.path.isdir(d) and d.endswith('.gdb'):
            if arcpy.Exists(os.path.join(d, 'scores')) == False:
                print(d)
                to_score.append((d, mode))

C:\wfrc\ato\scenario\Cycling\13400_south_bike_lane.gdb
C:\wfrc\ato\scenario\Cycling\4700_south_bike_lane.gdb
C:\wfrc\ato\scenario\Driving\new_2700_west_overpass.gdb
C:\wfrc\ato\scenario\Transit\box_elder_express.gdb


In [ ]:
# score unscored networks

# if testing, use a subset of centroids
#if test:
#    centroids = r"shp\taz_wfrc.gdb\taz_centroids_sample"


for scenario in to_score:
    gdb, mode = scenario
    ato.skim(
        gdb = gdb,
        mode = mode,
        skim_matrix = os.path.join(gdb, r"skim_matrix")
    )
    ato.score(
        gdb = gdb
    )

If the loop above fails due to an invalid network, you can rebuild the network by running `network.build()` on the full network dataset path, e.g. `network.build(r'scenario\Transit\ogden_local_bus.gdb\NetworkDataset\NetworkDataset_ND')`

In [31]:
# add land use projects
to_score = list()
scenario_folder = os.path.join('scenario', 'land_use')
for file in os.listdir(scenario_folder):
    d = os.path.join(base_path, scenario_folder, file)
    if os.path.isdir(d) and d.endswith('.gdb'):
        if arcpy.Exists(os.path.join(d, 'scores')) == False:
            print(d)
            to_score.append(d)

C:\wfrc\ato\scenario\land_use\ogden_cbd.gdb


In [68]:
#score land use projects
for gdb in to_score:

    ato.score(
        gdb = gdb,
        skim_matrix = os.path.join(baseline_gdb, r"skim_driving"), 
        job_per_hh = 1.808755,
        out_table = os.path.join(gdb, r"ato_driving")
    )
    
    ato.score(
        gdb = gdb,
        skim_matrix = os.path.join(baseline_gdb, r"skim_cycling"), 
        job_per_hh = 1.808755,
        out_table = os.path.join(gdb, r"ato_cycling")
    )
        
    ato.score(
        gdb = gdb,
        skim_matrix = os.path.join(baseline_gdb, r"skim_transit"), 
        job_per_hh = 1.808755,
        out_table = os.path.join(gdb, r"ato_transit")
    )
    
    ato.comp(gdb = gdb) # create composite by combining ato scores per mode
    
    # calculate difference between scenario ATO and baseline
    ato.diff(
        baseline = r'baseline.gdb\ato_comp', 
        scenario = os.path.join(gdb, 'ato_comp'),
        out_table = os.path.join(gdb, 'scores')
    )

Scores written to C:\wfrc\ato\scenario\land_use\ogden_cbd.gdb\ato_driving
Network ATO: 625012823.0
Scores written to C:\wfrc\ato\scenario\land_use\ogden_cbd.gdb\ato_cycling
Network ATO: 92704904.0
Scores written to C:\wfrc\ato\scenario\land_use\ogden_cbd.gdb\ato_transit
Network ATO: 45464197.0
Composite ATO 763181924.0 written to ato_comp
Scenario score: 1487432.0


## Tabulate Scores Across Scenarios

The cells below read in the scores from all scored scenarios and combined into a single summary table (`scenario\scenario_scores.csv`)

Scores for individual projects can be extracted from the `scores_summary` table within each file geodatabase.

In [ ]:
summary_long = pd.DataFrame(columns = ['Name', 'Mode', 'hh', 'jobs', 'ato'])

vals = {
    "Name":"Baseline",
    "Mode": "Auto",
    "hh": baseline_auto['accessible_hh'].sum(),
    "jobs": baseline_auto['accessible_jobs'].sum(),
    "ato": baseline_auto['ato'].sum()
}
summary_long = summary_long.append(vals, ignore_index=True)

vals = {
    "Name": "Baseline",
    "Mode": "Transit",
    "hh": baseline_transit['accessible_hh'].sum(),
    "jobs": baseline_transit['accessible_jobs'].sum(),
    "ato": baseline_transit['ato'].sum()
}
summary_long = summary_long.append(vals, ignore_index=True)

vals = {
    "Name": "Baseline",
    "Mode": "Cycling",
    "hh": baseline_cycling['accessible_hh'].sum(),
    "jobs": baseline_cycling['accessible_jobs'].sum(),
    "ato": baseline_cycling['ato'].sum()
}
summary_long = summary_long.append(vals, ignore_index=True)

In [ ]:
scenarios = list()

for mode in ['Driving', 'Transit', 'Cycling', 'Land_Use']:
    scenario_folder = os.path.join('scenario', mode)
    for file in os.listdir(scenario_folder):
        d = os.path.join(scenario_folder, file)
        if os.path.isdir(d) and d.endswith('.gdb'):
            if arcpy.Exists(os.path.join(d, 'scores')) == True:
                scenarios.append({
                                "name": file[:-4],
                                "mode": mode,
                                "gdb_path": d
                            })

In [ ]:
for scenario in scenarios:
    
    arr = arcpy.da.TableToNumPyArray(scenario['gdb_path'] + r'\scores_summary', field_list)
    scores = pd.DataFrame(arr, columns = field_list)
    
    mode = scenario['mode']
    
    if mode == 'Driving':
        baseline = r'baseline.gdb\ato_driving'
    elif mode == 'Transit':
        baseline = r'baseline.gdb\ato_transit'
    elif mode == 'Cycling':
        baseline = r'baseline.gdb\ato_cycling'
    elif mode == 'Land_Use':
        baseline = r'baseline.gdb\ato_comp'
    
    df = pd.merge(
        baseline, 
        scores, 
        on='CO_TAZID', 
        how="inner",
        suffixes=("_before", "_after")
    )
    df['diff_hh'] = df['accessible_hh_after'] - df['accessible_hh_before']
    df['diff_jobs'] = df['accessible_jobs_after'] - df['accessible_jobs_before']
    df['diff_ato'] = df['ato_after'] - df['ato_before']
    vals = {
        "Name": scenario['name'],
        'Mode': mode,
        "hh": df['diff_hh'].sum(),
        "jobs": df['diff_jobs'].sum(),
        "ato": df['diff_ato'].sum()
    }
    
    summary_long = summary_long.append(vals, ignore_index=True)

In [ ]:
summary_long

In [ ]:
summary_long.to_csv(r'scenario\scenario_scores.csv')